# Data Processing

Qidu Fu

**Contents**
- [0 Import libraries](#0_import_libraries)
- [1 Load data](#1-load-data)
    - [1.1 Get stack exchange data](#11-get-stack-exchange-data)
    - [1.2 Get SEAME data](#12-get-seame-data-development-dataset)
    - [1.3 Get ASCEND data](#13-get-ascend-data)
- [2 Process the stack exchange data: the title column](#2-process-the-stack-exchange-data-the-title-column)
- [3 Process the stack exchange data: the tags and title column](#3-process-the-stack-exchange-tags-and-title-columns)
- [4 Get basic information: cleaned stack exchange dataset](#4-get-basic-information-cleaned-stack-exchange-dataset)

## 0 Import libraries



In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ssl
import datasets
import re

## 1 Load data

### 1.1 Get stack exchange data
Observations
- 1. Not NULL values in the data
- 2. The dataset include 12401 records/rows and 3 columns

In [2]:
# The stack exchange data is from stack exchange API

def get_stack_data(path):
    """
    Load the stack exchange data
    param path: str: path to the stack exchange data
    return: pd.DataFrame: the stack exchange data
    """
    return pd.read_csv(path)

STACK_DF = get_stack_data('private/stack_exchange_all_questions.csv')
print(STACK_DF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12041 entries, 0 to 12040
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   question_id  12041 non-null  int64 
 1   title        12041 non-null  object
 2   tags         12041 non-null  object
dtypes: int64(1), object(2)
memory usage: 282.3+ KB
None


In [3]:
STACK_DF.head()

,question_id,title,tags
0,59815,My translation of Li Bai&#39;s 《三五七言》,"translation, poetry"
1,53654,What do these characters on an antique mural p...,"character-identification, traditional-characte..."
2,59823,Help in translating Li Bai&#39;s 《月下独酌&#183;其二》,"translation, poetry"
3,59821,purpose of using 了 with 要不,grammar
4,59652,Why does the character 的 is pronounced differe...,"pronunciation, songs"


### 1.2 Get SEAME data (development dataset)
Observations
- 1. Not NULL values in the data
- 2. The dataset include 5321 records/rows and 3 columns

In [4]:
# The SEAME DATASET IS FROM: 
# Zhiping Zeng, Yerbolat Khassanov, Van Tung Pham, Haihua Xu, Eng Siong Chng, 
# and Haizhou Li, “On the End-to-End Solution to Mandarin-English Code-switching 
# Speech Recognition,” arXiv preprint arXiv:1811.00241, 2018.

def get_seame_data(path):
    """
    This function reads the SEAME dataset from the given path 
    and returns a pandas dataframe.
    param path: str: path to the SEAME dataset
    return: pd.DataFrame: SEAME dataset
    """
    # Ignore SSL certificate errors
    ssl._create_default_https_context = ssl._create_unverified_context
    # Read the file without specifying column names
    df = pd.read_csv(path, sep='\t', header=None)
    # Reset SSL certificate to default
    ssl._create_default_https_context = ssl._create_default_https_context
    # Split the first column into 'ID' and 'sentence'
    df[['ID', 'sentence']] = df[0].str.split(pat=' ', n=1, expand=True)
    # Drop the original column
    df = df.drop(columns=[0])
    return df

SEAME_DF = get_seame_data('https://raw.githubusercontent.com/zengzp0912/' + 
                            'SEAME-dev-set/refs/heads/master/dev_sge/text')

In [5]:
SEAME_DF.head()

,ID,sentence
0,nc15m-08nc15mbp_0101-00190-00481,hello hello 可 以
1,nc15m-08nc15mbp_0101-01044-01130,往 下 一 点
2,nc15m-08nc15mbp_0101-02185-02444,那 个 wave 比 较 慢 一 点 啊
3,nc15m-08nc15mbp_0101-02503-02571,okay 可 以
4,nc15m-08nc15mbp_0101-02838-02963,讲 多 一 点 咯 可 以


In [6]:
SEAME_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5321 entries, 0 to 5320
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        5321 non-null   object
 1   sentence  5321 non-null   object
dtypes: object(2)
memory usage: 83.3+ KB


In [7]:
# save it to a csv file for future use
SEAME_DF.to_csv('private/seame.csv', index=False)

### 1.3 Get ASCEND data
Observations
- 1. Not NULL values in the data
- 2. The dataset include 9869 records/rows and 3 columns

In [8]:
# The third dataset is ASCEND
# The ASCEND DATASET IS FROM:
# Lovenia, H., Cahyawijaya, S., Winata, G., Xu, P., Xu, Y., Liu, Z., Frieske, 
# R., Yu, T., Dai, W., Barezi, E. J., Chen, Q., Ma, X., Shi, B., & Fung, 
# P. (2022, June). ASCEND: A spontaneous Chinese-English dataset 
# for code-switching in multi-turn conversation. Proceedings of the 
# Language Resources and Evaluation Conference, 7259–7268. European 
# Language Resources Association. https://aclanthology.org/2022.lrec-1.788

def get_ascend_data():
    """
    Load the ASCEND dataset
    param: None
    return: pd.DataFrame: the ASCEND training and development datasets
    """
    # Load dataset but remove audio feature
    data = datasets.load_dataset('CAiRE/ascend')

    # Convert dataset to a Pandas DataFrame
    ascend_tr_df = data['train'].to_pandas()
    # Use the development set for now
    ascend_dev_df = data['validation'].to_pandas()

    # Keep only the required columns
    ascend_tr_df = ascend_tr_df[["id", "transcription", "topic"]]
    ascend_dev_df = ascend_dev_df[["id", "transcription", "topic"]]

    return ascend_tr_df, ascend_dev_df
# Load ASCEND dataset with selected columns
ASCEND_DF, ASCEND_DV_DF = get_ascend_data()
ASCEND_DV_DF.head()


,id,transcription,topic
0,00000,嗯,technology
1,00001,小朋友我回想一下when i was in elementary school,technology
2,00002,like year three,technology
3,00003,i have a phone but it's not a smart phone at t...,technology
4,00004,就是其实主要的功能就是打电话然后跟parents联系,technology


In [9]:
# ASCEND_DV_DF.info()

In [10]:
ASCEND_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9869 entries, 0 to 9868
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             9869 non-null   object
 1   transcription  9869 non-null   object
 2   topic          9869 non-null   object
dtypes: object(3)
memory usage: 231.4+ KB


In [11]:
# save it to a csv file for future use
ASCEND_DF.to_csv('private/ascend.csv', index=False)

## 2 Process the stack exchange data: the title column
Data processing pipeline aims at cleaning to the dataset to the level of the 
SEAME dataset from the published Chinese-English code-switching [paper](https://arxiv.org/abs/1811.00241). 
- 1. Randomly sample 20 rows/records multiple times for uncleanliness checking
- 2. Check URLs (non present)
    - URLs are not relevant to the code-switching. 
- 3. Replace special symbols/html encoding for their actual characters, such as &quot as "
    - These special symbols are not HTML representations of the actual characters. 
    The replacement correct that.
- 4. Remove non-Chinese-or-English characters, such as number+letters, 
    letters+numbers
    - This project focuses on Chinese-English code-switching. 
    Non-Chinese-or-English characters are irrelevant, which may be gibberish
    from website scraping. 

In [12]:
def sample_rows(df, n=20):
    """
    This function samples n rows from the given dataframe.
    param df: pd.DataFrame: dataframe to sample from
    param n: int: number of rows to sample
    return: pd.DataFrame: sampled dataframe
    """
    return df.sample(n)

sample_rows(STACK_DF)

,question_id,title,tags
7616,27825,Details of TOCFL online?,vocabulary
3980,41987,Can someone break down the sentence structure ...,"grammar, sentence-structure, simplified-charac..."
2786,48827,Are these sentences interchangeable?,"grammar, sentence-structure"
1418,54427,What Does It Mean When 过来 Indicates A Return T...,direction-complement
1593,53831,Is Wiktionary right that 女奴 meant &quot;cat&qu...,meaning
11127,8776,The Shi Shi Shi Shi Shi poem?,pinyin
6090,36050,Difference between 还/更 in comparasion phrases,comparison
3238,45324,减: subtracting from or reducing to?,translation
9201,22500,Ancient forms of characters,"characters, traditional-characters, classical-..."
6119,35938,What is the difference between 需 and 須?,"meaning, difference"


In [13]:
# Function to check all the rows with a given pattern
def check_pattern_non_regex(df, pattern):
    """
    Check all the rows with a given pattern
    param df: pd.DataFrame: dataframe to check
    param pattern: str: pattern to check
    return: pd.DataFrame: dataframe with rows containing the pattern
    """
    return df[df.str.contains(pattern, na=False)]

def check_pattern_regex(df, pattern):
    """
    Check all the rows with a given pattern
    param df: pd.DataFrame: dataframe to check
    param pattern: str: pattern to check
    return: pd.DataFrame: dataframe with rows containing the pattern
    """
    return df[df.str.contains(pattern, na=False, regex=True)]

In [14]:
# Check if url is present the stack exchange data
check_pattern_regex(STACK_DF['title'], r'https?://\S+')
# This will return all the rows with a url in the title column

Series([], Name: title, dtype: object)

In [15]:

# Function to replace A with B using regex
def replace_text(text, pattern, replace_with):
    """
    Replace a pattern in a text with a given string
    param text: str: input text
    param pattern: str: pattern to replace
    param replace_with: str: string to replace the pattern with
    return: str: text with the pattern replaced
    """
    return re.sub(pattern, replace_with, text)

# Function to replace " &quot;" with '"' and "&#39;" with "'"
def replace_quot(text):
    """
    Replace &quot; with "
    param text: str: input text
    return: str: text with &quot or &#39;; replaced with "
    """
    text = replace_text(text, "&quot;", '"')
    text = replace_text(text, "&#39;", "'")
    return text

# Test the function
print(replace_quot('This is a &quot;test&quot; string'))

This is a "test" string


In [16]:
# Apply the function to the title quotes in the stack exchange dataset
STACK_DF['title'] = STACK_DF['title'].apply(replace_quot)
print(STACK_DF.iloc[10710, 1])
print(STACK_DF.iloc[11633, 1])
print(STACK_DF.iloc[2547, 1])

What is the difference: "和小猫一样可爱" and "像小猫一样可爱"
How do you say 'dates back to' in Mandarin?
一 [份/个/项] 工作??? What is the difference?


In [17]:
print(STACK_DF.iloc[8067]['title'])
print(STACK_DF.iloc[4462]['title'])

Why 批 means "criticize" and "a batch" at the same time?
What's the exact difference between 住 and 居?


In [18]:
# Function to remove 's, 'll, and others
def remove_apostrophe(text):
    """
    Remove 's, 'll, 've, 're, 'd, 'm, 'em from text
    param text: str: input text
    return: str: text with 's, 'll, 've, 're, 'd, 'm, 'em removed
    """
    text = re.sub(r"'ll", "", text)
    text = re.sub(r"'ve", "", text)
    text = re.sub(r"'re", "", text)
    text = re.sub(r"'d", "", text)
    text = re.sub(r"'m", "", text)
    text = re.sub(r"'em", "", text)
    text = re.sub(r"'t", "", text)
    text = re.sub(r"n't", "not", text)
    text = re.sub(r"'", "", text)
    text = re.sub(r'"s', "", text)
    text = re.sub(r'"ll', "", text)
    text = re.sub(r'"ve', "", text)
    text = re.sub(r'"re', "", text)
    text = re.sub(r'"d', "", text)
    text = re.sub(r'"m', "", text)
    text = re.sub(r'"em', "", text)
    text = re.sub(r'"t', "", text)
    text = re.sub(r'"', "", text)
    text = re.sub(r"’s", "", text)
    text = re.sub(r"’ll", "", text)
    text = re.sub(r"’ve", "", text)
    text = re.sub(r"’re", "", text)
    text = re.sub(r"’d", "", text)
    text = re.sub(r"’m", "", text)
    text = re.sub(r"’em", "", text)
    text = re.sub(r"’t", "", text)
    text = re.sub(r"n’t", "not", text)
    text = re.sub(r"’", "", text)
    text = re.sub(r"‘s", "", text)
    text = re.sub(r"‘ll", "", text)
    text = re.sub(r"‘ve", "", text)
    text = re.sub(r"‘re", "", text)
    text = re.sub(r"‘d", "", text)
    text = re.sub(r"‘m", "", text)
    text = re.sub(r"‘em", "", text)
    text = re.sub(r"‘t", "", text)
    text = re.sub(r"n‘t", "not", text)
    text = re.sub(r"‘", "", text)
    return text

# remove_apostrophe("I'm going to the store. He'll be there.")
# remove_apostrophe(STACK_DF.iloc[192, 1])

In [19]:
print(STACK_DF.iloc[192, 1])
# STACK_DF['title'] = STACK_DF['title'].apply(remove_apostrophe)
print(STACK_DF.iloc[192, 1])

Understanding and interpretation of 李白's 《玉阶怨》
Understanding and interpretation of 李白's 《玉阶怨》


In [20]:
# Resample again to check other signs of data uncleanliness
sample_rows(STACK_DF)

,question_id,title,tags
1215,54931,旁边那个大点儿的杯子是我的 — how to correctly break down th...,"grammar, meaning, sentence-structure"
8257,27585,Can 再说 be used in a written context?,"meaning-in-context, phrase"
5734,36835,Sign on the edge of a cross-country ski trail ...,translation
3094,47827,是 assessing an overall situation,"grammar, meaning, usage, difference, sentence-..."
10911,10003,Some self-sufficient Chinese phrases. Help me?,pinyin
1419,6729,Is there any difference between 冰淇淋 and 冰激凌,"word-choice, difference"
8164,28015,How does 也 work in 宁愿冻死也不穿?,"meaning-in-context, adverbs"
5549,31062,What are the characters on this seal? (Charact...,"characters, character-identification, seal"
10590,12708,节期: ji&#233;qī vs. ji&#233;q&#237;,pronunciation
1050,55398,What types of non-fountain pens or pencils (no...,handwriting


In [21]:
print('STACK_DF.iloc[7598, 1]: ')
print(STACK_DF.iloc[7598, 1])
print('STACK_DF.iloc[4374, 1]: ')
print(STACK_DF.iloc[4374, 1])
print('STACK_DF.iloc[1041, 1]: ')
print(STACK_DF.iloc[1041, 1])
print('STACK_DF.iloc[1230, 1]: ')
print(STACK_DF.iloc[1230, 1])
print('STACK_DF.iloc[6224, 1]: ')
print(STACK_DF.iloc[6224, 1])
print('STACK_DF.iloc[664, 1]: ')
print(STACK_DF.iloc[664, 1])
print('STACK_DF.iloc[5366, 1]: ')
print(STACK_DF.iloc[5366, 1])

STACK_DF.iloc[7598, 1]: 
Is this [Ivanka Trump] Chinese proverb real? —“Those who say it can not be done, should not interrupt those doing it”
STACK_DF.iloc[4374, 1]: 
What semantic notions underlie "to think; to contemplate", with "but; however; nevertheless" with "only"?
STACK_DF.iloc[1041, 1]: 
Should the blank in 他毕业以后很快 _____ 找到了一个理想的工作 be filled with 才, 可, or 便?
STACK_DF.iloc[1230, 1]: 
Can you say someone is a ”非常熟的人“？
STACK_DF.iloc[6224, 1]: 
“管你是[surname] + [name]还是[same surname] + [different name]”
STACK_DF.iloc[664, 1]: 
D&#224;o d&#233; and d&#224;od&#233;. Difference between each word separately? And when combined together as a compound word?
STACK_DF.iloc[5366, 1]: 
好客	vs 亲切 - both mean hospitable?


In [22]:
# replace D&#224;o d&#233; and d&#224;od&#233 with correct terms
STACK_DF['title'] = STACK_DF['title'].apply(replace_text, args=('D&#224;o d&#233;', 'Dao de'))
STACK_DF['title'] = STACK_DF['title'].apply(replace_text, args=('d&#224;od&#233;', 'daode'))
STACK_DF['title'] = STACK_DF['title'].apply(replace_text, args=('d&#233;', 'de'))
print(STACK_DF.iloc[664, 1])
print(STACK_DF.iloc[2907, 1])

Dao de and daode. Difference between each word separately? And when combined together as a compound word?
Why is 肯德基 (Kěndejī) = Kentucky Fried Chicken not called 肯德鸡 (Kěndejī) when it's pronounced the same and 鸡 means "chicken"?


In [23]:
# Check underscores in the title column
# check_pattern_regex(STACK_DF['title'], r'_+')

In [24]:
# remove underscores from the title column
# STACK_DF['title'] = STACK_DF['title'].apply(lambda x: re.sub(r'_+', ' ', x))
# check_pattern_regex(STACK_DF['title'], r'_+')

In [25]:
# Resample again to check other signs of data uncleanliness
sample_rows(STACK_DF)

,question_id,title,tags
4113,30102,What pinyin combinations spell out English words?,pinyin
2181,51790,"How should ""燭之死"" be understood in the followin...","meaning-in-context, classical-chinese"
11938,1814,What's the difference between “问一下”，“查询”， “询问”？,"word-choice, usage"
9737,17631,When should I use 经验 and when should I use 经历?,grammar
3845,42417,What is so problematic about the homophones 预订...,"usage, difference"
3714,42784,What do new Chinese speakers sound like to nat...,spoken
2199,51731,How is 目前世界上除了多媒体形式的书以外，还有用纸印刷的书 illogical?,grammar
7195,31289,Placement of adjectives with verbs?,"translation, grammar"
5000,39639,"In 立下了汗马功劳, how should I understand 立下了 (""set ...","meaning, chengyu"
3386,43819,What is the character missing in 「好吃jing」?,word-requests


In [26]:
import html

# Define the function to decode all HTML and Unicode encodings
def decode_html_entities(text):
    # Step 1: Decode named and decimal numeric entities (e.g., &#224;, &#xE0;)
    text = html.unescape(text)
    
    # Step 2: Decode hexadecimal numeric entities (e.g., &#xE0;)
    text = re.sub(r'&#x([0-9A-Fa-f]+);', lambda x: chr(int(x.group(1), 16)), text)
    
    # Step 3: Decode decimal numeric entities (e.g., &#224;)
    text = re.sub(r'&#(\d+);', lambda x: chr(int(x.group(1))), text)

    # Step 4: Decode Unicode escaped sequences like \uXXXX manually
    text = re.sub(r'\\u([0-9A-Fa-f]{4})', lambda x: chr(int(x.group(1), 16)), text)

    return text

# Apply the decoding function to the 'title' column
STACK_DF['title'] = STACK_DF['title'].apply(decode_html_entities)

# Apply the decoding function to the 'tags' column if needed
STACK_DF['tags'] = STACK_DF['tags'].apply(decode_html_entities)


In [27]:
print('STACK_DF.iloc[7488, 1]: ')
print(STACK_DF.iloc[7488, 1])
print('STACK_DF.iloc[9375, 1]: ')
print(STACK_DF.iloc[9375, 1])
print('STACK_DF.iloc[6587, 1]: ')
print(STACK_DF.iloc[6587, 1])
print('STACK_DF.iloc[335, 1]: ')
print(STACK_DF.iloc[335, 1])

STACK_DF.iloc[7488, 1]: 
Why is 把 in 我去洗把脸 (wǒ qù xǐ bǎ liǎn) = I'm going to wash my face?
STACK_DF.iloc[9375, 1]: 
Blessed bat pun: 蝙蝠→__福
STACK_DF.iloc[6587, 1]: 
Why is 弄 (nòng) in 他终于弄明白了?
STACK_DF.iloc[335, 1]: 
How did Middle Chinese "hæwk" evolve into Mandarin "xué"?


In [28]:
check_pattern_non_regex(STACK_DF['title'], 'q249')

Series([], Name: title, dtype: object)

In [29]:
# remove numbers/words with numbers from the title column
print(STACK_DF.iloc[3638, 1])
# STACK_DF['title'] = STACK_DF['title'].apply(lambda x: re.sub(r'\w*\d\w*', '', x))
print(STACK_DF.iloc[3638, 1])
print(check_pattern_non_regex(STACK_DF['title'], 'q249'))

Is there any distinction between 二百 (èrbǎi) and 两百 (liǎng bǎi) which both mean "two hundred"?
Is there any distinction between 二百 (èrbǎi) and 两百 (liǎng bǎi) which both mean "two hundred"?
Series([], Name: title, dtype: object)


In [30]:
# Function to remove non English or Chinese characters
def remove_non_en_ch(text):
    """
    Remove non-English or Chinese characters
    param text: str: input text
    return: str: text with non-English or Chinese characters removed
    """
    return re.sub(r'[^\x00-\x7F\u4E00-\u9FA5]', '', text)

# print(remove_non_en_ch(STACK_DF.iloc[11613, 1]))

In [31]:
print(STACK_DF.iloc[11613, 1])
# STACK_DF['title'] = STACK_DF['title'].apply(remove_non_en_ch)
print(STACK_DF.iloc[11613, 1])

Is 去 (qù) pronunced /tɕʰu/ or /tɕʰy/?
Is 去 (qù) pronunced /tɕʰu/ or /tɕʰy/?


In [32]:
# Resample again to check other signs of data uncleanliness
sample_rows(STACK_DF)

,question_id,title,tags
9687,18057,Artist seal and meaning,"translation, transliteration, seal"
2695,50141,Use 微积分 to mean 'tiny'?,meaning-in-context
9626,3120,Learn the characters at the same time as the l...,"characters, learning"
228,54045,Game 四加一 or 四紮 rules or origin unknown,"simplified-characters, traditional-characters"
1940,51934,Why is 奇数 pronounced jīshù and not qíshù?,"etymology, pronunciation"
3720,42770,Please help with the meaning of seal calligrap...,"translation, seal, calligraphy"
9054,23182,Asian ornate bakelite? bowl that is prob 20th ...,translation
4573,40562,What does 我三谁了 mean in 我也不知道，我刷四绝朋友告诉我有人刷喇叭骂我小...,"translation, meaning-in-context"
9072,22581,"Is 中文亲属名称 the correct translation for ""Family ...",translation
1728,53380,How to literally translate this: 这对于那些想看又买不起书的...,"translation, sentence-structure"


In [33]:
# check to remove rows with null values on the title column
STACK_DF = STACK_DF.dropna(subset=['title'])
STACK_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12041 entries, 0 to 12040
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   question_id  12041 non-null  int64 
 1   title        12041 non-null  object
 2   tags         12041 non-null  object
dtypes: int64(1), object(2)
memory usage: 282.3+ KB


## 3 Process the stack exchange: tags and title columns
Data processing pipeline aims at cleaning to the dataset to the level of the 
SEAME dataset from the published Chinese-English code-switching [paper](https://arxiv.org/abs/1811.00241). 
The tags columns are much cleaners as compared to the previous column
- Remove empty strings in the title column and replace with NaN values
- Remove rows with NaN values in the title column
- Rename the column to be `text` and `topic`
- Save the stack exchange data to a CSV to avoid repeatition processing

In [34]:
# check for non English or Chinese  characters
check_pattern_non_regex(STACK_DF['tags'], r'[^\x00-\x7F\u4E00-\u9FA5]')

Series([], Name: tags, dtype: object)

In [35]:
# check for punctuations if other than ,
check_pattern_non_regex(STACK_DF['tags'], r'[^\w\s,]')

1        character-identification, traditional-characte...
5        translation, character-identification, seal, c...
11                               classical-chinese, poetry
13             translation, character-identification, seal
17        translation, word-choice, terminology, loanwords
                               ...                        
12023                               word-choice, cantonese
12026                                          word-choice
12031                            verbs, meaning-in-context
12032                             mandarin, mainland-china
12038                 characters, character-identification
Name: tags, Length: 4342, dtype: object

In [36]:
# remove - from the tags columns
# STACK_DF['tags'] = STACK_DF['tags'].apply(lambda x: re.sub(r'-', ' ', x))
check_pattern_non_regex(STACK_DF['tags'], r'-')

1        character-identification, traditional-characte...
5        translation, character-identification, seal, c...
11                               classical-chinese, poetry
13             translation, character-identification, seal
17        translation, word-choice, terminology, loanwords
                               ...                        
12023                               word-choice, cantonese
12026                                          word-choice
12031                            verbs, meaning-in-context
12032                             mandarin, mainland-china
12038                 characters, character-identification
Name: tags, Length: 4342, dtype: object

In [37]:
sample_rows(STACK_DF)

,question_id,title,tags
686,57825,Can 一府两院 be replaced with 三院?,word-choice
7952,28275,Why is inital /ŋ/ considered hick-ish?,"pronunciation, topolect"
10584,12700,"When to use ""一下儿""",grammar
684,57883,Is the first day of a week Monday or Sunday in...,culture
10685,12200,Difference between alveolars and alveolo-palat...,cantonese
4006,40899,Is it possible to combine 起来 and 住在？,"grammar, usage, sentence-structure, word-order..."
2219,16434,"Why is ""他在椅子上坐"" wrong?",grammar
5283,38781,What is 女斗士 and when can I use this?,translation
2749,48946,"relation between ""疏"" and ""上命"" (clarifying mean...","translation, classical-chinese"
5560,10238,"Is 目 suitable for daily conversations, or shou...","usage, difference"


In [38]:
print('STACK_DF.iloc[4763, 1]: ')
print(STACK_DF.iloc[4763, 2])
# replace - with space in the tags column
STACK_DF['tags'] = STACK_DF['tags'].apply(lambda x: re.sub(r'-', ' ', x))
print(STACK_DF.iloc[4763, 2])

STACK_DF.iloc[4763, 1]: 
traditional-characters, software, fonts
traditional characters, software, fonts


In [39]:
# rename the columns, and only select the title and tags columns
STACK_DF = STACK_DF.rename(columns={'title': 'text', 'tags': 'topic'})[['text', 'topic']]
STACK_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12041 entries, 0 to 12040
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    12041 non-null  object
 1   topic   12041 non-null  object
dtypes: object(2)
memory usage: 188.3+ KB


In [40]:
# remove empty strings from the text column
print(STACK_DF.iloc[4957, ])
STACK_DF['text'] = STACK_DF['text'].apply(lambda x: x.strip())
print(STACK_DF.iloc[4957, ])
# convert empty strings to NaN
STACK_DF['text'] = STACK_DF['text'].replace('', np.nan)
STACK_DF.info()

text     HSK6的第101题（缩写）请求反馈：富有爸爸和会花钱的儿子
topic             writing critique, hsk
Name: 4957, dtype: object
text     HSK6的第101题（缩写）请求反馈：富有爸爸和会花钱的儿子
topic             writing critique, hsk
Name: 4957, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12041 entries, 0 to 12040
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    12041 non-null  object
 1   topic   12041 non-null  object
dtypes: object(2)
memory usage: 188.3+ KB


In [41]:
# drop NaN values if title is empty
STACK_DF = STACK_DF.dropna(subset=['text'])
STACK_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12041 entries, 0 to 12040
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    12041 non-null  object
 1   topic   12041 non-null  object
dtypes: object(2)
memory usage: 188.3+ KB


In [42]:
# save the cleaned df to a csv file
STACK_DF.to_csv('private/stack_exchange_cleaned.csv', index=False)
print('STACK_DF saved to private/stack_exchange_cleaned.csv')

STACK_DF saved to private/stack_exchange_cleaned.csv


In [43]:
# save the random 1000 rows of the cleaned df to a csv file for sample 
random_sample_idx = np.random.choice(STACK_DF.shape[0], 1000, replace=False)
STACK_DF.iloc[random_sample_idx].to_csv('data/stack_exchange_cleaned_sample.csv', index=False)
print('Random 1000 rows of STACK_DF saved to data/stack_exchange_cleaned_sample.csv')

Random 1000 rows of STACK_DF saved to data/stack_exchange_cleaned_sample.csv


## 4 Get basic information: cleaned stack exchange dataset

The cleaned dataset contains **12,039 entries** with two columns:

- **text**: Posts from Stack Exchange (12,039 non-null values, `object` string).
- **topic**: General discourse domain of each post (12,039 non-null values, `object` string).

Additional information
- The `topic` column still needs processing, such as word tokenization or others 
for topic modeling/clustering. At the moment, it has 3500 combinations using the
`value_counts` function.
- The text column may still needs further processing, such as word tokenization 
and others. 

In [44]:
CLEANED_STACK_DF = pd.read_csv('private/stack_exchange_cleaned.csv')
CLEANED_STACK_DF.head()

,text,topic
0,My translation of Li Bai's 《三五七言》,"translation, poetry"
1,What do these characters on an antique mural p...,"character identification, traditional characte..."
2,Help in translating Li Bai's 《月下独酌·其二》,"translation, poetry"
3,purpose of using 了 with 要不,grammar
4,Why does the character 的 is pronounced differe...,"pronunciation, songs"


In [45]:
CLEANED_STACK_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12041 entries, 0 to 12040
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    12041 non-null  object
 1   topic   12041 non-null  object
dtypes: object(2)
memory usage: 188.3+ KB


In [46]:
CLEANED_STACK_DF['topic'].value_counts()

translation                                                            974
grammar                                                                601
meaning                                                                464
meaning in context                                                     238
word choice                                                            217
                                                                      ... 
terms of address, formal                                                 1
meaning in context, etymology, character identification                  1
idioms, colloquialisms                                                   1
character identification, traditional characters, seal, old chinese      1
translation, style                                                       1
Name: topic, Length: 3502, dtype: int64